In [ ]:
import pandas as pd
from typing import Dict, Any


def carregar_e_selecionar_dados(
    caminho_arquivo: str, colunas_importantes: list
) -> pd.DataFrame:
    """
    Carrega o arquivo CSV e seleciona as colunas importantes.

    Args:
        caminho_arquivo (str): O caminho para o arquivo CSV de entrada.
        colunas_importantes (list): Uma lista de nomes de colunas a serem selecionadas.

    Returns:
        pd.DataFrame: O DataFrame com as colunas selecionadas.
    """
    try:
        df = pd.read_csv(caminho_arquivo, sep=";")
        return df[colunas_importantes].copy()
    except FileNotFoundError:
        print(f"Erro: O arquivo não foi encontrado no caminho '{caminho_arquivo}'")
        return pd.DataFrame()


def _processar_dt_notific(df: pd.DataFrame) -> None:
    """
    Processa a coluna 'DT_NOTIFIC', convertendo-a para o tipo datetime e renomeando.

    Args:
        df (pd.DataFrame): O DataFrame a ser processado.
    """
    df.dropna(subset=["DT_NOTIFIC"], inplace=True)
    df["DT_NOTIFIC"] = pd.to_datetime(
        df["DT_NOTIFIC"], format="%Y-%m-%d", errors="coerce"
    )
    df.rename(columns={"DT_NOTIFIC": "DATA_NOTIFICACAO"}, inplace=True)


def _processar_colunas_mapeamento(
    df: pd.DataFrame, colunas: list, mapeamento: Dict[Any, str]
) -> None:
    """
    Processa colunas que requerem preenchimento de valores ausentes e mapeamento.

    Args:
        df (pd.DataFrame): O DataFrame a ser processado.
        colunas (list): Uma lista de nomes de colunas para processar.
        mapeamento (dict): O dicionário de mapeamento para os valores da coluna.
    """
    for col in colunas:
        df[col].fillna(9.0, inplace=True)
        df[col] = df[col].replace(mapeamento)


def _processar_evolucao(df: pd.DataFrame, mapeamento: Dict[Any, str]) -> None:
    """
    Processa a coluna 'EVOLUCAO', lidando com valores ausentes e mapeamento especial.

    Args:
        df (pd.DataFrame): O DataFrame a ser processado.
        mapeamento (dict): O dicionário de mapeamento para os valores da coluna.
    """
    df["EVOLUCAO"].fillna(9.0, inplace=True)
    df["EVOLUCAO"] = df["EVOLUCAO"].replace(3.0, 2.0)
    df["EVOLUCAO"] = df["EVOLUCAO"].replace(mapeamento)


def limpar_e_transformar_dados(df: pd.DataFrame) -> pd.DataFrame:
    """
    Orquestra a limpeza e transformação dos dados do DataFrame.

    Args:
        df (pd.DataFrame): O DataFrame a ser limpo e transformado.

    Returns:
        pd.DataFrame: O DataFrame com os dados limpos.
    """
    if df.empty:
        return pd.DataFrame()

    mapeamento_sim_nao = {1.0: "SIM", 2.0: "NAO", 9.0: "IGNORADO"}
    mapeamento_evolucao = {1.0: "CURA", 2.0: "OBITO", 9.0: "IGNORADO"}

    _processar_dt_notific(df)
    _processar_colunas_mapeamento(
        df, ["UTI", "VACINA", "VACINA_COV"], mapeamento_sim_nao
    )
    _processar_evolucao(df, mapeamento_evolucao)

    return df


def salvar_dados(df: pd.DataFrame, caminho_arquivo: str) -> None:
    """
    Salva o DataFrame processado em um novo arquivo CSV.

    Args:
        df (pd.DataFrame): O DataFrame a ser salvo.
        caminho_arquivo (str): O caminho para o arquivo de saída.
    """
    if not df.empty:
        df.to_csv(caminho_arquivo, index=False)
        print(f"Dados salvos com sucesso em '{caminho_arquivo}'")


def main():
    """
    Função principal que orquestra todo o processo de ETL (Extração, Transformação e Carga).
    """
    # Definição dos caminhos dos arquivos
    caminho_arquivo_entrada = "../data/01_raw/INFLUD25-11-08-2025.csv"
    caminho_arquivo_saida = "../data/03_final/dataset.csv"

    # Lista de colunas a serem extraídas
    colunas_importantes = ["DT_NOTIFIC", "UTI", "EVOLUCAO", "VACINA", "VACINA_COV"]

    # 1. Carregar e Extrair dados
    df = carregar_e_selecionar_dados(caminho_arquivo_entrada, colunas_importantes)

    # 2. Limpar e Transformar dados
    df_final = limpar_e_transformar_dados(df)

    # 3. Salvar os dados processados
    salvar_dados(df_final, caminho_arquivo_saida)


if __name__ == "__main__":
    main()